<a href="https://colab.research.google.com/github/step-code01/Scene-CaptureAssistant/blob/main/SubjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from google.colab import files
from google.colab.patches import cv2_imshow

uploaded_files = files.upload()
image = cv2.imread(list(uploaded_files.keys())[0])
if image is None:
  print("Error: could not load image")
  exit()

image = cv2.resize(image,(640,480))

##Subject Detection

In [ ]:
saliency = cv2.saliency.StaticSaliencyFineGrained_create()

(success,saliency_map) = saliency.computeSaliency(image)

saliency_map = (saliency_map*255).astype("uint8")
_,saliency_mask = cv2.threshold(saliency_map,180,255,cv2.THRESH_BINARY)

'''"If a pixel is ≥ 180, set it to 255 (white). Otherwise, set it to 0 (black)." '''

# Find contours (blobs) of salient regions
contours,_ = cv2.findContours(saliency_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

# If found any blobs, assume the largest one is the main subject
if contours:
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    subject_center = (x + w // 2, y + h // 2)

    visual = image.copy()
    cv2.rectangle(visual, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.circle(visual, subject_center, 5, (0, 0, 255), -1)
    print("Subject Detection: ")
    cv2_imshow(visual)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f" Subject detected at (x={subject_center[0]}, y={subject_center[1]})")

else:
    print(" No strong subject detected — photo may lack a clear focal point.")